In [48]:
import os 
%pwd

'e:\\RenalTumor'

In [49]:
# os.chdir('../')
%pwd

'e:\\RenalTumor'

In [50]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [51]:
from src.KidneyDiseaseClassifier.constants import *
from src.KidneyDiseaseClassifier.utils.common import read_yaml,create_directories


In [52]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir, 
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone" # <-- Add this second part
        )          
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE 
        )

        return training_config

In [53]:
# File: .../components/training.py

import tensorflow as tf
from pathlib import Path

tf.config.run_functions_eagerly(True)

class Training:
    def __init__(self, config: "TrainingConfig"):
        self.config = config
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        self.train_generator = None
        self.valid_generator = None

    def get_base_model(self):
        """Loads the updated base model from the specified path."""
        print("Loading the updated and compiled base model for training...")
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        """Creates training and validation data generators."""
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=20,
                shear_range=0.1,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def _save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        print(f"Final trained model saved to: {path}")

    def train(self):
        """Executes the final model training loop."""
        steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        print("--- Starting Final Model Training ---")
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=validation_steps
        )

        self._save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [54]:

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
    
except Exception as e:
    raise e

[Thu, 03 Jul 2025 23:34:31: INFO: common: Loading Yaml file succesfully.]
[Thu, 03 Jul 2025 23:34:31: INFO: common: Loading Yaml file succesfully.]
[Thu, 03 Jul 2025 23:34:31: INFO: common: Directory created at: artifacts]
[Thu, 03 Jul 2025 23:34:31: INFO: common: Directory created at: artifacts\training]
[Thu, 03 Jul 2025 23:34:31: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Loading the updated and compiled base model for training...
[Thu, 03 Jul 2025 23:34:31: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
--- Starting Final Model Training ---
Epoch 1/5
311/311 ━━━━━━━━━━━━━━━━━━━━ 5050s 16s/step - accuracy: 0.6783 - loss: 6.0938 - val_accuracy: 0

In [55]:
# import sys
# import tensorflow
# import keras

# print("--- KERNEL ENVIRONMENT DIAGNOSTICS ---")
# print(f"Python Executable: {sys.executable}")
# print(f"TensorFlow Version: {tensorflow.__version__}")
# print(f"Keras Version: {keras.__version__}")
# print("------------------------------------")